# Understanding MSE vs. Asymmetric Loss Functions

## Introduction
This interactive visualization helps us understand the difference between Squared Error (SE) and Asymmetric Loss Functions, with a specific focus on Asymmetric Glucose Loss from Hops_1 in DSC40a. These loss functions serve different purposes in machine learning, particularly when the cost of errors in different directions is not equal.

## The Loss Functions

### Squared Error (SE)
SE is one of the most common loss function, defined as:

$$\text{MSE}(y, \hat{y}) = (y - \hat{y})^2$$


Key characteristics:
- Symmetric: Penalizes overestimates and underestimates equally
- Always non-negative
- Grows quadratically with the difference between predicted ($\hat{y}$) and true ($y$) values


### Asymmetric Glucose Loss
This specialized loss function is defined as:

$$\text{AsymmetricGlucoLoss}(y, \hat{y}) = \begin{cases} 3(y - \hat{y})^2 & \text{if } \hat{y} < y \\ (y - \hat{y})^2 & \text{if } \hat{y} \geq y \end{cases}$$


Key characteristics:
- Asymmetric: Penalizes underestimates three times more heavily than overestimates
- Particularly useful when certain erros are should be punished more.
- Still maintains quadratic growth like MSE, but with different scaling factors

In [2]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np

In [3]:
#data
y = np.linspace(-3, 3, 200)
y_hat = 0
mse = (y - y_hat)**2

In [4]:
###################################################
################# RUN THIS CODE ###################
###################################################
# you will see an output link that looks like this: 
# Dash app running on http://127.0.0.1:8050/ 
# at the bottom of this cell
###################################################
################ CLick that link ##################
###################################################
# Dash app
app = Dash(__name__)
app.layout = html.Div([
    html.H1('Understanding Asymmetric Loss Functions', 
            style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 30}),
    
    html.Div([
        dcc.Graph(id='loss-plot'),
        
        html.Div([
            html.Label('Asymmetry Parameter (a):', 
                      style={'fontSize': 16, 'marginBottom': 10}),
            dcc.Slider(
                id='a-slider',
                min=-5,
                max=5,
                step=0.1,
                value=0,
                marks={i: {'label': str(i), 'style': {'fontSize': 14}} 
                       for i in range(-5, 6)},
                updatemode='drag'
            )
        ], style={'width': '80%', 'margin': 'auto', 'marginTop': 20}),
        
        html.Div([
            html.H3('How to interpret:', 
                    style={'color': '#2c3e50', 'marginTop': 30}),
            html.Ul([
                html.Li('Blue solid line: Standard Squared Error (SE) loss', 
                       style={'marginBottom': 10}),
                html.Li('Purple dashed line: Asymmetric Loss - watch how one side bends!', 
                       style={'marginBottom': 10}),
                html.Li('When a > 0: The right side bends up, making the model more sensitive to underestimation', 
                       style={'marginBottom': 10}),
                html.Li('When a < 0: The left side bends up, making the model more sensitive to overestimation', 
                       style={'marginBottom': 10}),
                html.Li('When a = 0: Both losses match perfectly', 
                       style={'marginBottom': 10})
            ], style={'fontSize': 14})
        ], style={'width': '80%', 'margin': 'auto', 'marginTop': 20})
    ])
], style={'fontFamily': 'Arial, sans-serif', 'margin': 20})

@app.callback(
    Output('loss-plot', 'figure'),
    Input('a-slider', 'value')
)
def update_figure(a):
    # Calculate asymmetric loss - only one side bends at a time
    if a >= 0:
        # Positive a: right side bends up (more sensitive to underestimation)
        asym_loss = np.where(y > y_hat,
                            (1 + a) * (y - y_hat)**2,  # Right side bends
                            mse)                        # Left side stays fixed
    else:
        # Negative a: left side bends up (more sensitive to overestimation)
        asym_loss = np.where(y < y_hat,
                            (1 - a) * (y - y_hat)**2,  # Left side bends
                            mse)                        # Right side stays fixed
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=y, 
        y=mse, 
        mode='lines', 
        name='SE',
        line=dict(color='#2980b9', width=2.5)
    ))
    
    fig.add_trace(go.Scatter(
        x=y, 
        y=asym_loss, 
        mode='lines',
        name='Asymmetric Loss',
        line=dict(color='#8e44ad', width=2.5, dash='dash')
    ))
    
    fig.update_layout(
        title=dict(
            text=f'Loss Functions Comparison (a = {a:.2f})',
            x=0.5,
            font=dict(size=20)
        ),
        xaxis_title=dict(text='True Value (y)', font=dict(size=14)),
        yaxis_title=dict(text='Loss', font=dict(size=14)),
        xaxis=dict(range=[-3, 3], gridcolor='#eee'),
        yaxis=dict(range=[0, 10], gridcolor='#eee'),
        plot_bgcolor='white',
        height=500,
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(255,255,255,0.8)'
        )
    )
    
    return fig

if __name__ == '__main__':
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


## Using the Interactive Visualization

### What You Can Observe
1. **When a = 0**: The asymmetric loss (purple dashed line) perfectly overlaps with MSE (blue solid line), showing the baseline case.

2. **When a > 0**: 
   - The right side of the purple line (y > 0) rises higher than MSE
   - This represents increased penalties for underestimation
   - Useful when underestimating is more dangerous than overestimating

3. **When a < 0**:
   - The left side of the purple line (y < 0) rises higher than MSE
   - This represents increased penalties for overestimation
   - Useful when overestimating is more dangerous than underestimating

### Practical Applications

1. **Medical Monitoring**
   - In glucose monitoring, underestimating blood sugar levels is more dangerous than overestimating
   - The asymmetric loss with positive 'a' helps create models that err on the side of caution

2. **Resource Planning**
   - In supply chain management, underestimating demand might be more costly than overestimating
   - The asymmetric loss can be tuned to reflect these real-world cost differences

3. **Risk Assessment**
   - In financial modeling, underestimating risk might be more dangerous than overestimating it
   - The asymmetric loss can help create more conservative risk models

## Mathematical Insights

1. **Continuity**
   - Both functions are continuous across all inputs
   - The asymmetric loss maintains smoothness at y = ŷ, despite different coefficients

2. **Gradients**
   - MSE has a symmetric gradient around ŷ
   - Asymmetric loss has different gradients for underestimation vs. overestimation
   - This difference leads to different optimization behaviors during model training


Dash app running on http://127.0.0.1:8050/
